In [2]:
import sys
import os
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns
from cobra import Reaction


#Change working dir first, ty ChatGPT, much loves
cwd = os.getcwd()
# Split the path into a list of directories
directories = cwd.split(os.sep)
# Remove the last two directories from the list
directories = directories[:-2]
# Join the directories back into a path
new_cwd = os.sep.join(directories)
# Change the current working directory to the new path
os.chdir(new_cwd)

sys.path.append("./src")

import model_initialize as model_init
import model_manipulation  as mm


Checklist:
- add wild type script to this script //
- Change names to reflect WT and trans //
- Incorporate graphs for both WT and trans//

- Check WT FVA @ 1000 PPFD to check CO2 intake and base CO2 script from there.



In [3]:
#This codeblock is to define some of the functions used for modelling

##UPDATED JULY 28 2023

##NOTES:
##Fixed wrong encoding of reactions for Malate dehydrogenase as well as NADP ME.



#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light
inf=1e6

def generate_constraint(model,reaction, name, lb, ub):
    reaction_fex = model.reactions.get_by_id(name).flux_expression
    constraint = model.problem.Constraint(reaction_fex, lb=lb, ub=ub)
    constraint.name = name + '_constraint'
    model.add_cons_vars

def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp


#This function is used to set the inputs to the model used. 
def define_model_medium(model, co2, o2, ppfd, 
                        medium_dir='./misc/photo_medium.csv', no3=inf, h2o=inf, h=inf, 
                        nh4=inf, pi=inf):
    model_photo_media = mm.read_medium_csv(medium_dir, model)
    model_photo_media['EX_no3(e)'] = no3
    model_photo_media['EX_h2o(e)'] = h2o
    model_photo_media['EX_h(e)'] = h
    model_photo_media['EX_nh4(e)'] = nh4
    model_photo_media['EX_co2(e)'] = co2
    model_photo_media['EX_o2(e)'] = o2
    model_photo_media['EX_photonVis(e)'] = ppfd
    model_photo_media['EX_pi(e)'] = pi
    #Set set model medium as model
#     print('Added model medium')
    return model_photo_media

    
def turn_off_cofac_cycles(model, inact_dir='./misc/leaf_inactivated.tsv'):
    file = csv.reader(open(inact_dir), delimiter='\t')
    leaf_inactive_rxns = list()
    for rows in file:
        row_m = str()
        row_bs = str()
        for rxns in rows:
            row_m += str(rxns) + "_M"
            row_bs += str(rxns) + "_BS"
        leaf_inactive_rxns.append(row_m)
        leaf_inactive_rxns.append(row_bs)
        
    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns:
            rxns.bounds = (0,0)
#     print('Successfully turned off cofactor-cycling reactions')

    
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")
def add_tissue_constraints(model):
    #For input fluxes for light, we will set the flux ratio to 10:1 to reflect the anatomical proportions of our model ()
    
    BS_photon_import = model.reactions.PRISM_white_LED_BS
    M_photon_import = model.reactions.PRISM_white_LED_M

    #Set photon flux ratio to 10:1
    photon_flux = mm.set_fix_flux_ratio({M_photon_import.id:10, BS_photon_import.id:1},model)
    model.add_cons_vars(photon_flux)

    
    #UPDATE: Change CO2 intake to the M Cell instead rather than set a ratio, which is a better assumption overall. Assume na lang that external gasses are assimilated
    #Via the M cell.
    #From Morrison et al 2005 -- Lateral diffusion of Gases is unlikely to support photosynthesis due to the
    #assimilation of diffused CO2 in tissues prior to BS//
    model.reactions.CO2tex_BS.bounds = (0,0)
    model.reactions.O2tex_BS.bounds = (0,0)
    
    #UPDATE: This assumption does not hold considering that recent transcriptomic analysis confirms that 
    #the bundle sheath is involved in the assimilation of inorganic nutrients, including nitrogen (nitrates/ammonia), and 
    #Sulfates. In turn, this will be implemented by simply setting the exchanges to the M cell to 0. (Hua et al, 2021)
    model.reactions.SO3tex_M.bounds = (0,0)
    model.reactions.SO4tex_M.bounds = (0,0)
    model.reactions.NH4tex_M.bounds = (0,0)
    model.reactions.NO3tex_M.bounds = (0,0)
    
    #Model will also constraint H2O input to BS cell only as it is also assumed that BS tissue in rice is specialized for H2O transport (Hua et al. 2021)
    #There is a demand reaction naman for H2O for the M cell which is not connected to the BS H2Otex
    #Restrict H2O transport to be unidirectional from the BS cell
    model.reactions.H2Otex_M.bounds = (0, 0)
    model.reactions.h2o_pd.bounds = (-inf, 0)
    
    #need to turn off HCO import as the model incorrectly transfers fixed HCO to the BS cell via the common pool compartment
    model.reactions.HCO3tex_M.bounds = (0,0)
    model.reactions.HCO3tex_BS.bounds = (0,0)
    
    #Turn off extracellular Glycine transport 
    model.reactions.GLYtex_M.bounds = (0,0)
    model.reactions.GLYtex_BS.bounds = (0,0)
    
    #Turn off other Demand reactions that may serve as sinks for the model except DM_Phloem_BS (Which represents the output of photoassimilate thru the BS cell
    model.reactions.DM_Phloem_M.bounds = (0,0)
    model.reactions.Straw_Biomass_M.bounds = (0,0)
    model.reactions.Straw_Biomass_BS.bounds = (0,0)
    model.reactions.Coleoptile_Biomass_M.bounds = (0,0)
    model.reactions.Coleoptile_Biomass_BS.bounds = (0,0)
    model.reactions.DM_Phloem_BS.bounds = (0, inf)
    

def add_enzyme_constraints(model, 
                           wt_pepc = 0, 
                           wt_mdh = 11.18, 
                           wt_nadp_me = 0.14, 
                           wt_ppdk=0.31,
                          wt_CA=7.5):
    
    
    # #This code block contains constraints specific for enzyme rate constraints
    #This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
    #fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
    #At 0. For reversible reactions the lower bounds are set to the same value
    
    
    #PEPC constraint (Reaction id: PPCc)
    #Need to constrain it to 0 since reaction is only detected in Vascular tissue
    pepc_BS = model.reactions.PPCc_BS
    pepc_M = model.reactions.PPCc_M
    
    pepc_BS.bounds = (0,0)
    pepc_M.bounds = (0,0)

    #PPDK constraints (Reaction id: PPDKs) (note that this is found in the chloroplast?) 
    #Not detected via immunolocalization but enzyme activity is detected

    ppdks_BS = model.reactions.PPDKs_BS
    ppdks_M = model.reactions.PPDKs_M
    ppdkc_BS = model.reactions.PPDKc_BS
    ppdkc_M = model.reactions.PPDKc_M
    wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                             + ppdks_M.flux_expression
                                             + ppdkc_BS.flux_expression
                                             + ppdkc_M.flux_expression, 
                                             lb = 0, ub = wt_ppdk)
    wt_ppdks_cons.name = 'wt_ppdks_cons'
    model.add_cons_vars(wt_ppdks_cons)
    
    
    #Malate Dehydrogenase 
    #Only mitochondrial in WT Rice M cells
    model.reactions.MDHs_M.bounds = (0,0)
    model.reactions.MDHc_M.bounds = (0,0)
    model.reactions.MDHx_M.bounds = (0,0)
    model.reactions.MDHs_BS.bounds = (0,0)
    model.reactions.MDHc_BS.bounds = (0,0)
    model.reactions.MDHx_BS.bounds = (0,0)
    
    #Add constraints to MDHm
    mdhm_M = model.reactions.MDHm_M
    
    wt_mdh_cons = model.problem.Constraint(mdhm_M.flux_expression,
                                           lb= -wt_mdh, ub=wt_mdh)
    wt_mdh_cons.name = "wt_mdh_cons"
    model.add_cons_vars(wt_mdh_cons)

    #NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
    #Let's see if I can force it to have a small amount of flux 
    mdh2s_M = model.reactions.MDH2s_M
    mdh2s_BS = model.reactions.MDH2s_BS
    mdh2c_M = model.reactions.MDH2s_M
    mdh2c_BS = model.reactions.MDH2s_BS


    wt_nadpme_cons = model.problem.Constraint(mdh2s_M.flux_expression
                                             + mdh2s_BS.flux_expression
                                              + mdh2c_M.flux_expression
                                              + mdh2c_BS.flux_expression,
                                             lb= 0, ub=wt_nadp_me)
    wt_nadpme_cons.name = "wt_nadpme_cons"
    model.add_cons_vars(wt_nadpme_cons)


    #I should add constraints for Carbonic Anhydrase
    #I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
    #Flux is reversible so constraints are bi-directional
    #This should be revised considering that it allows reversible reactions  and an abnormally high flux thru carbonic anhydrase, which shouldn't be the case

    hco3es_m = model.reactions.HCO3Es_M.flux_expression
    hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
    hco3em_m = model.reactions.HCO3Em_M.flux_expression
    hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
    hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
    hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

    ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                       + hco3es_bs + hco3ec_bs + hco3em_bs,
                                      lb = -wt_CA, ub = wt_CA)
    ca_cons.name = 'Carbonic_anhydrase_constraint'
    model.add_cons_vars(ca_cons)


    #Rbcl constaints
    #Retrieve flux expressions oof each RBCl reaction
    rbpc_M = model.reactions.RBPCs_M.flux_expression
    rbpc_BS = model.reactions.RBPCs_BS.flux_expression
    rbpo_M = model.reactions.RBPOs_M.flux_expression
    rbpo_BS = model.reactions.RBPOs_BS.flux_expression

    #Constraint such that it is limited to 132 umol m-2 s-1
    rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)
    rbcl_vcmax_cons.name='rbcl_vcmax_cons'
    model.add_cons_vars(rbcl_vcmax_cons)
    #Constraints for rbcl flux such that v_c/v_o = 3 or higher.
    rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                         - 1*(rbpc_M + rbpc_BS),
                                         lb=0,ub=1000)
    rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
    model.add_cons_vars(rbcl_vcvo)

    #Turn off the RBPC2s reactions since we already defined the constraints above
    model.reactions.RBPC2s_M.bounds = (0,0)
    model.reactions.RBPC2s_BS.bounds = (0,0)
    
    
    
    #What if I simply constrained that of the M cell one to 3:1?
    #This constraint is pretty good actually. 
    #This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
    #wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.
# #This code block is to set a constraint such that M-to-BS cell NGAM ratio is 10-to-1 
# #Similar to what Moreno-Villena et al (2022) had done 

#This function takes two arguments: the model and the maximal  ppfd input to the system
def add_ngam_cons(model, ppfd): 
    ngam_atp_m = mm.get_rxn(model, 'ngam_atp_c_M')
    ngam_atp_bs = mm.get_rxn(model, 'ngam_atp_c_BS')
    ngam_atp_m.bounds = (0,inf)
    ngam_atp_bs.bounds = (0,inf)
    ngam_ratio = mm.set_fix_flux_ratio({ngam_atp_m.id:10, ngam_atp_bs.id:1}, model)
    ngam_ratio.name = 'ngam_BS/M_ratio'
    model.add_cons_vars(ngam_ratio)

    #Retrieve NGAM reactions
    ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c_M')
    ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s_M')
    ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m_M')
    ngam_nadphox_c_BS = mm.get_rxn(model, 'ngam_nadphox_c_BS')
    ngam_nadphox_s_BS = mm.get_rxn(model, 'ngam_nadphox_s_BS')
    ngam_nadphox_m_BS = mm.get_rxn(model, 'ngam_nadphox_m_BS')


    #Set Fixed fluxes
    nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
    nadphox_c_s_M.name = "nadphox_cs_ratio_M"
    nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
    nadphox_s_m_M.name = "nadphox_sm_ratio_M"

    nadphox_c_s_BS = mm.set_fix_flux_ratio({ngam_nadphox_c_BS.id:1, ngam_nadphox_s_BS.id:1},model)
    nadphox_c_s_BS.name = "nadphox_cs_ratio_BS"
    nadphox_s_m_BS = mm.set_fix_flux_ratio({ngam_nadphox_s_BS.id:1, ngam_nadphox_m_BS.id:1}, model)
    nadphox_s_m_BS.name = "nadphox_sm_ratio_BS"

    #Add constraints
    model.add_cons_vars(nadphox_c_s_M)
    model.add_cons_vars(nadphox_s_m_M)
    model.add_cons_vars(nadphox_c_s_BS)
    model.add_cons_vars(nadphox_s_m_BS)

    #Retrieve flux expressionns
    fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
    fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
    fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)

    fex_nadphox_c_BS =  mm.get_flux_exp(model, ngam_nadphox_c_BS)
    fex_nadphox_s_BS =  mm.get_flux_exp(model, ngam_nadphox_s_BS)
    fex_nadphox_m_BS =  mm.get_flux_exp(model, ngam_nadphox_m_BS)

    fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_m)
    fex_atp_c_BS =  mm.get_flux_exp(model, ngam_atp_bs)

    #Set the constraint between ATP:NADPH NGAM to 3:1
    nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M
                                                       + fex_nadphox_c_BS + fex_nadphox_s_BS + fex_nadphox_m_BS) 
                                         - 1*(fex_atp_c_M + fex_atp_c_BS),
                                         lb=0,ub=0)
    nadphox_atpase.name = "nadphox_atpase_ratio"
    model.add_cons_vars(nadphox_atpase)
    #Compute NGAM value and add constraint as a lower bound/upper bound to model
    ngam_value = compute_ngam_atp(ppfd)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    ngam_cons.name = 'NGAM_ATP_constraint'
    model.add_cons_vars(ngam_cons)
    
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above

def print_summary(model, sample_fluxes_df):
    print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
    print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
    print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])
    print('RBPC2s_M', sample_fluxes['RBPC2s_M'], 'RBPC2s_BS', sample_fluxes['RBPC2s_BS'])
    print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC BS', sample_fluxes['PPCc_BS'])
    print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec_M'], 'Carbonic Anhydrase (Cytosolic) BS', sample_fluxes['HCO3Ec_BS'])
    print('NADP-ME M', sample_fluxes['MDHys_M'], 'NADP-ME BS', sample_fluxes['MDHys_BS'])
    print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
    print('Phloem M: ', sample_fluxes['DM_Phloem_M'], 'Phloem BS', sample_fluxes['DM_Phloem_BS'])
    print('co2 consumption M', sample_fluxes['CO2tex_M'], 'co2 consumption BS', sample_fluxes['CO2tex_BS'])
    print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
    print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
    print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
    print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])
    print('ATP synthesis (stromal) M', sample_fluxes['ATPSs_M'], 'ATP synthase (mit) M', sample_fluxes['ATPSm_M'])
    pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
    pd_abs_flux = 0
    for pds in pd_rxn:
        pd_abs_flux += abs(sample_fluxes[pds.id])
    
    print('pd_abs_flux: ', pd_abs_flux)
    
#initialize list of transgenic reactions to add  to model

def add_trans_reactions(model):
    '''
    This function is used to add a number of new tissue-specific reactions that were not present in the
    original model to facilitate modelling of the transgenic C4 rice
    '''
    trans_list = list()
    #Transgenic PEPC copy
    #PEPC = Chloroplastic in M & V (rxn id: PPCc)
    trans_ppcs = Reaction('trans_PPCs_M')
    trans_ppcs.name = "Phosphoenolpyruvate carboxylase, plastidic (Transgenic)"
    
    pep_s0 = model.metabolites.pep_s0
    hco3_s0 = model.metabolites.hco3_s0
    oaa_s0 = model.metabolites.oaa_s0
    pi_s0 = model.metabolites.pi_s0


    #Add metabolites, bounds, and subsystem
    trans_ppcs.add_metabolites({hco3_s0:-1, pep_s0:-1, oaa_s0:1, pi_s0:1})
    trans_ppcs.bounds= model.reactions.PPCc_M.bounds
    trans_ppcs.subsystem = model.reactions.PPCc_M.subsystem

    trans_list.append(trans_ppcs)


    #Transgenic PPDK Copy
    trans_ppdks_m = Reaction('trans_PPDKs_M')
    trans_ppdks_m.add_metabolites(model.reactions.PPDKs_M.metabolites)
    trans_ppdks_m.bounds = model.reactions.PPDKs_M.bounds
    trans_ppdks_m.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_ppdks_bs = Reaction('trans_PPDKs_BS')
    trans_ppdks_bs.add_metabolites(model.reactions.PPDKs_BS.metabolites)
    trans_ppdks_bs.bounds = model.reactions.PPDKs_BS.bounds
    trans_ppdks_bs.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

    trans_list.append(trans_ppdks_m)
    trans_list.append(trans_ppdks_bs)

    #Transgenic NADP-ME
    #NADP-ME = Mitochondrial in M
    trans_nadp_me = Reaction('trans_MDH2m_M')

    #retrieve reactants
    mal_m0 = model.metabolites.get_by_id('mal-L_m0')
    nadp_m0 = model.metabolites.nadp_m0
    co2_m0 = model.metabolites.co2_m0
    nadph_m0 = model.metabolites.nadph_m0
    pyr_m0 = model.metabolites.pyr_m0

    #Add to rxn
    trans_nadp_me.add_metabolites({mal_m0:-1, nadp_m0:-1, co2_m0:1, nadph_m0:1, pyr_m0:1})
    #Add bounds
    trans_nadp_me.bounds=(-inf, inf)

    trans_list.append(trans_nadp_me)


#     #Malate Dehydrogenase, mitochondrial (M cell)
#     trans_MDHm_M = Reaction('trans_MDHm_M')
#     trans_MDHm_M.name = 'Malate Dehydrogenase, Mitochondrial'
#     trans_MDHm_M.add_metabolites(model.reactions.MDHm_M.metabolites)
#     trans_MDHm_M.subsystem = model.reactions.MDHm_M.subsystem
#     trans_MDHm_M.notes['SUBSYSTEM'] = trans_MDHm_M.subsystem

#     trans_list.append(trans_MDHm_M)

#     #Malate dehydrogenase, plastidic (M cell)
#     trans_MDHs_M = Reaction('trans_MDHs_M')
#     trans_MDHs_M.name = 'Malate Dehydrogenase, Plastidic'
#     trans_MDHs_M.add_metabolites(model.reactions.MDHs_M.metabolites)
#     trans_MDHs_M.subsystem = model.reactions.MDHs_M.subsystem
#     trans_MDHs_M.notes['SUBSYSTEM'] = trans_MDHs_M.subsystem
#     trans_list.append(trans_MDHs_M)

#     #Malate dehydrogenase, plastidic(BS Cell)
#     trans_MDHs_BS = Reaction('trans_MDHs_BS')
#     trans_MDHs_BS.name = 'Malate Dehydrogenase, Plastidic'
#     trans_MDHs_BS.add_metabolites(model.reactions.MDHs_BS.metabolites)
#     trans_MDHs_BS.subsystem = model.reactions.MDHs_BS.subsystem
#     trans_MDHs_BS.notes['SUBSYSTEM'] = trans_MDHs_BS.subsystem

#     trans_list.append(trans_MDHs_BS)


    #Trans CA
    #Cytosolic in M
    trans_hco3ec_M = Reaction('trans_hco3ec_M')
    trans_hco3ec_M.name = 'carbonic anhydrase, cytosolic'
    trans_hco3ec_M.add_metabolites(model.reactions.HCO3Ec_M.metabolites)
    trans_hco3ec_M.bounds = model.reactions.HCO3Ec_M.bounds

    trans_hco3ec_M.subsystem = model.reactions.HCO3Ec_M.subsystem
    trans_list.append(trans_hco3ec_M)


    #Bulk add to model
    model.add_reactions(trans_list)
    
    model.repair()
####ADDING TRANS CONSTRAINTS

def add_trans_constraints(model,
                         trans_pepc_rates = 7.01,
                         trans_ppdks_rates = 3.66,
                         trans_mdh_rates = 152.87,
                         trans_nadp_me_rates = 0.60,
                         trans_CA_rates = 8):
    '''
    This function is used to add another layer of constraints to parametize model based on the
    Enzyme reaction rates assayed from Ermakova et al (2021) where the locations are based on the 
    each of the transgenic enzyme's tissue-specific localizations. 
    '''
    
    #PEPC constraint
    wt_PPCc_M = mm.get_rxn(model, 'PPCc_M')
    wt_PPCc_BS = mm.get_rxn(model, 'PPCc_BS')
    trans_PPCs_M = mm.get_rxn(model, 'trans_PPCs_M')                           
    trans_PEPC_cons = model.problem.Constraint(trans_PPCs_M.flux_expression
                                            +wt_PPCc_BS.flux_expression 
                                            + wt_PPCc_M.flux_expression, 
                                            lb = 0, ub = trans_pepc_rates)

    model.add_cons_vars(trans_PEPC_cons)

    #PPDK constraint
    trans_PPDKs_M  = mm.get_rxn(model, 'trans_PPDKs_M')
    trans_PPDKs_BS = mm.get_rxn(model, 'trans_PPDKs_BS')
    wt_PPDKs_M = mm.get_rxn(model, 'PPDKs_M')
    wt_PPDKs_BS = mm.get_rxn(model, 'PPDKs_BS')
    
    trans_PPDKs_cons = model.problem.Constraint( 
        trans_PPDKs_BS.flux_expression + trans_PPDKs_M.flux_expression 
        +wt_PPDKs_BS.flux_expression + wt_PPDKs_M.flux_expression, 
                                             lb = 0, ub = trans_ppdks_rates)
    trans_PPDKs_cons.name = 'trans_ppdks_cons'
    model.add_cons_vars(trans_PPDKs_cons)
    


    #Malate Dehydrogenase Constraints
    trans_MDHm_M = mm.get_rxn(model, 'MDHm_M')
    trans_MDHs_M = mm.get_rxn(model, 'MDHs_M')
    trans_MDHs_BS = mm.get_rxn(model, 'MDHs_BS')
    
    #Change bounds to reflect the Trans state (Based on Immunoblotting)
    trans_MDHm_M.bounds = (-inf, inf)
    trans_MDHs_M.bounds = (-inf, inf)
    trans_MDHs_BS.bounds = (-inf, inf)
    
    trans_mdh_cons =  model.problem.Constraint(
       trans_MDHm_M.flux_expression + 
        trans_MDHs_M.flux_expression + 
        trans_MDHs_BS.flux_expression, 
        lb= -trans_mdh_rates, ub=trans_mdh_rates)

    trans_mdh_cons.name = "trans_mdh_cons"
    model.add_cons_vars(trans_mdh_cons)

    #Add NADP-ME constraints
    trans_MDH2m_M = mm.get_rxn(model, 'trans_MDH2m_M')
    wt_MDH2s_M = mm.get_rxn(model, 'MDH2s_M')
    wt_MDH2s_BS = mm.get_rxn(model, 'MDH2s_BS')
    
    
    
    trans_nadpme_cons = model.problem.Constraint(
        trans_MDH2m_M.flux_expression + 
        wt_MDH2s_M.flux_expression + 
        wt_MDH2s_BS.flux_expression,
        lb= 0, ub=trans_nadp_me_rates)
    
    trans_nadpme_cons.name = "trans_nadpme"
    model.add_cons_vars(trans_nadpme_cons)

    #Add carbonic anhydrase constraints

    trans_hco3ec_M = mm.get_rxn(model, 'trans_hco3ec_M')
    wt_hco3ec_M = mm.get_rxn(model, 'HCO3Ec_M')
    wt_hco3em_M = mm.get_rxn(model, 'HCO3Em_M')
    wt_hco3es_M = mm.get_rxn(model, 'HCO3Es_M')
    wt_hco3ec_BS = mm.get_rxn(model, 'HCO3Ec_BS')
    wt_hco3em_BS = mm.get_rxn(model, 'HCO3Em_BS')
    wt_hco3es_BS = mm.get_rxn(model, 'HCO3Es_BS')
    
    trans_ca_cons = model.problem.Constraint(trans_hco3ec_M.flux_expression + 
                                             wt_hco3es_M.flux_expression + 
                                             wt_hco3ec_M.flux_expression + 
                                             wt_hco3em_M.flux_expression + 
                                             wt_hco3es_BS.flux_expression + 
                                             wt_hco3ec_BS.flux_expression + 
                                             wt_hco3em_BS.flux_expression,
                                      lb = -trans_CA_rates, ub = trans_CA_rates)
    trans_ca_cons.name = 'Trans_CA_cons'
    model.add_cons_vars(trans_ca_cons)
    model.repair()
    


#Read 2-cell model
wt_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")
trans_model = cobra.io.read_sbml_model("./model/ios2164_2cell.xml")

wt_model.solver = 'gurobi'
trans_model.solver = 'gurobi'


trans_model
add_trans_reactions(trans_model)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-09


#Add transgenic reactions to model localized to specific compartment

Transgenic (Line 29 - Ermakova et al)
PEPC = Chloroplastic in M & V (rxn id: PPCc)
PPDK = Chloroplastic in M and BS (rxn id: PPDKs)
NADP-ME = Mitochondrial in M (rxn id: MDHys)
MDH = Chloroplastic in M and BS; Mitochondrial in M  (rxn id: MDH2s, MDH2c)
CA = Cytosolic in M (rxn id: hco3ec)

In WT:
PEPC = Cytosolic in V
PPDK = No signal
NADP-ME = No signal
MDH = Mitochondrial in M 
CA = No signal

Difference:
PEPC = Only in M
PPDK = Chloroplastic M & BS
NADP-ME = Only in M
MDH = Chloroplastic in M/BS
CA = Cytosolic in M


My approach would be to add two linear constraints corresponding to the WT and the transgenic Enzyme reaction rates, respectively. The first layer of constraints represents WT conditions while the second layer constraints represents the Transgenic conditions wherein enzyme capacities would be increased relative to each compartment.


Part C. Generating Flux distributions for varying light levels (From 50 to 1500 ppfd)


In [6]:
#Clear out model first
test_model = wt_model

with test_model as m:
    ppfd = 1000
    test_model.medium = define_model_medium(m, co2=29, o2=2.26, ppfd=ppfd, h=, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m)
    add_tissue_constraints(m)
    add_enzyme_constraints(m)
    add_ngam_cons(m, ppfd)
    
    #Turn on all Demand reactions for protons and see how it goes
    for rxns in m.reactions:
        if "DM_h" in rxns.id:
            rxns.bounds = (0, inf)

    mm.get_rxn(m,'Straw_Biomass_M').objective_coefficient = 0
    mm.get_rxn(m,'Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(m, 'DM_Phloem_BS').objective_coefficient = 1

    test_model = wt_model.copy()
    
test_model.metabolites.get_by_id('h_c0').summary()

#This codeblock is used to generate solutions and check summaries for individual reactions/metabolites

Read LP format model from file /tmp/tmpj6a0_y2r.lp
Reading time = 0.02 seconds
: 3970 rows, 9882 columns, 42998 nonzeros


Percent,Flux,Reaction,Definition
0.39%,0.462,ACALDHc_M,acald_c0 + h2o_c0 + nad_c0 --> ac_c0 + 2.0 h_c0 + nadh_c0
27.67%,32.61,ASCOXc_M,4.0 ascb-L_c0 + o2_c0 --> 2.0 h2o_c0 + 4.0 h_c0 + 4.0 mhdascb_c0
0.80%,0.9402,ASCPO1c_M,2.0 ascb-L_c0 + h2o2_c0 --> 2.0 h2o_c0 + 2.0 h_c0 + 2.0 mhdascb_c0
0.34%,0.4021,ASPCLc_M,asp-L_c0 + h_c0 <=> ala-B_c0 + co2_c0
8.38%,9.87,CYOO6m_M,4.0 focytc_m0 + 8.0 h_m0 + o2_m0 --> 4.0 ficytc_m0 + 2.0 h2o_m0 + 4.0 h_c0
16.75%,19.74,CYORm_M,2.0 ficytc_m0 + 2.0 h_m0 + q8h2_m0 --> 2.0 focytc_m0 + 4.0 h_c0 + q8_m0
1.02%,1.206,DHPDc_M,56dura_c0 + h2o_c0 <=> cala_c0 + 3.0 h_c0
18.72%,22.07,FRORc_M,frnsol_c0 + nadp_c0 --> frnsl_c0 + h_c0 + nadph_c0
0.17%,0.1978,GAPDHc_M,g3p_c0 + nad_c0 + pi_c0 <=> 13dpg_c0 + h_c0 + nadh_c0
3.80%,4.479,GLYtx_M,gly_c0 + h_c0 <=> gly_x0 + h_x0


In [4]:
# This code block is to test whether a single iteration works. Maybe I should implement the kinetic constraints by Von Caemmerer to obtain proper 
#proper values? I think that would be a better approach, I think.
with wt_model as m1:
    ppfd = 1500                                                                                                                                                                                                                                                                                     
    m1.medium = define_model_medium(m1, co2=29, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
    turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
    add_tissue_constraints(m1)
    add_enzyme_constraints(m1)
    add_ngam_cons(m1, ppfd)

    #Change objective function to mature leaf
    m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(m1,'DM_Phloem_BS').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(m1).fluxes
    #Remove thermodynamically infeasible loops
    sample_fluxes = cobra.flux_analysis.loopless_solution(m1, sample_fluxes)
    sample_fluxes_df = sample_fluxes.to_frame()


print_summary(m1, sample_fluxes_df)



rbcl M cell:  33.64605425339737 rbcl BS cell:  1.2020508242815215
rbcl M cell (photorespiration) 9.198123322939267 rbcl BS cell (PR) 2.4179117029536967
vc/vo M: 3.657925978170702 vc/vo BS: 0.49714421863011304
RBPC2s_M 0.0 RBPC2s_BS 0.0
PEPC M 0.0 PEPC BS 0.0
Carbonic Anhydrase (Cytosolic) M 0.0 Carbonic Anhydrase (Cytosolic) BS -0.044738917769733946
NADP-ME M 0.0 NADP-ME BS 0.004525262141844698
Biomass M:  0.0 Biomass BS 0.0
Phloem M:  0.0 Phloem BS 0.4670033170118366
co2 consumption M 29.000000000000085 co2 consumption BS 0.0
o2 consumption M 0.0 o2 consumption BS 0.0
Photosystem II M 90.62237788644681 PSII BS 9.075447617395485
PSI M 143.43847700402898 PSI BS 14.33397625621471
PPFD M:  998.4918056587599 PPFD BS:  99.84918056587598
ATP synthesis (stromal) M 143.43847700402898 ATP synthase (mit) M 10.338076635951285
pd_abs_flux:  1.8585568931292036


In [5]:
co2_2cell_wt_phloem = pd.DataFrame(index=list(i.id for i in wt_model.reactions))

#Iterate from 0 to 29 CO2 assimilation

co2_range = np.linspace(0,29, 30)
for co2 in co2_range:
    
    #Generate instance of wt_model every iteration
    with wt_model as m1:
        #Set PPFD to 1500
        ppfd = 1500
        
        #Set medium to change photon flux and other things and re-add to wt_model
        m1.medium = define_model_medium(m1, co2=co2, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
        turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
        add_tissue_constraints(m1)
        add_enzyme_constraints(m1)
        #Add NGAM to wt_model
        add_ngam_cons(m1, ppfd)
    
        #Change objective function to mature leaf
        m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        while True:
            solution = m1.optimize()
            if solution.status == 'optimal':
                sample_fluxes = cobra.flux_analysis.pfba(m1).fluxes
                #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                sample_fluxes = cobra.flux_analysis.loopless_solution(m1, sample_fluxes).to_frame()
                print("CO2 assimilation:", co2, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                #Append to Dataframe
                co2_2cell_wt_phloem[co2] = sample_fluxes['fluxes']
                break


filepath = './flux_results/CO2_benchmark/FBA/WT/'
filename = 'CO2-benchmark-wt_model-Mature-Leaf-1500-ppfd'
mm.save_fba_matrix(filename, co2_2cell_wt_phloem, filepath)


#I don't think I need to implement the other Biomass-based wt_model anymore since I'm dealing with mature leaves instead

CO2 assimilation: 0.0  RBPC_M cell flux: 0.0
CO2 assimilation: 1.0  RBPC_M cell flux: 1.176145130871947
CO2 assimilation: 2.0  RBPC_M cell flux: 2.3184663420666785
CO2 assimilation: 3.0  RBPC_M cell flux: 3.4902524519980647
CO2 assimilation: 4.0  RBPC_M cell flux: 4.647374079125911
CO2 assimilation: 5.0  RBPC_M cell flux: 5.764651453352375
CO2 assimilation: 6.0  RBPC_M cell flux: 6.887466326702157
CO2 assimilation: 7.0  RBPC_M cell flux: 8.032338951050908
CO2 assimilation: 8.0  RBPC_M cell flux: 9.198260161463473
CO2 assimilation: 9.0  RBPC_M cell flux: 10.371298168280978
CO2 assimilation: 10.0  RBPC_M cell flux: 11.576599587380485
CO2 assimilation: 11.0  RBPC_M cell flux: 12.748454216768742
CO2 assimilation: 12.0  RBPC_M cell flux: 13.908471508036966
CO2 assimilation: 13.0  RBPC_M cell flux: 15.06269444875433
CO2 assimilation: 14.0  RBPC_M cell flux: 16.223998018900655
CO2 assimilation: 15.0  RBPC_M cell flux: 17.38530158904672
CO2 assimilation: 16.0  RBPC_M cell flux: 18.546605159192

In [11]:
#For Mature Leaf (Phloem as Objective function)
#Wild Type

#Define n and ppfd ranges for simulating n and light conditions respectively
ppfd_range = np.linspace(50, 1500, 10) #array([ 250.,  500.,  750., 1000., 1250., 1500.])
co2_range = np.linspace(0,29, 10)
#First initialize excel spreadsheet
filepath = './flux_results/CO2_benchmark/FBA/WT/'
filename = 'CO2|PPFD-benchmark-Phloem-EX-leaf.xls'
workbook = mm.create_xlsx(filename, filepath)


for ppfd in ppfd_range:

    co2_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in wt_model.reactions))

    for co2 in co2_range:
        
        #Generate instance of model every iteration
        with wt_model as m1:

            m1.medium = define_model_medium(m1, co2=co2, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf) #initialize model
            turn_off_cofac_cycles(m1)
            add_tissue_constraints(m1)
            add_enzyme_constraints(m1)
            add_ngam_cons(m1, ppfd)                #Set ATP NGAM value to linear PPFD formula

            #Change objective function to mature leaf
            m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
            m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
            m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

            #Optimize then check optimality
            while True:
                solution = m1.optimize()
                if solution.status == 'optimal':
                    sample_fluxes = cobra.flux_analysis.pfba(m1).fluxes
                    #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                    sample_fluxes = cobra.flux_analysis.loopless_solution(m1, sample_fluxes).to_frame()
                    print("PPFD:", ppfd, "CO2 flux:", sample_fluxes['fluxes']['CO2tex_M'], " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                    #Append to Dataframe
                    co2_solns_2cell_fba[co2] = sample_fluxes['fluxes']
                    break
                else:
                    continue
                
    #Write matrix to excel workbook
        
    co2_solns_2cell_fba.to_excel(workbook, sheet_name=str(ppfd))

workbook.close()

#Notes:
#Complete dataframe does not save
#I think I fixed it now, I saved each instance to the wrong column lol



Successfully generated CO2|PPFD-benchmark-Phloem-EX-leaf.xls-20230728-20:48 in ./flux_results/CO2_benchmark/FBA/WT/CO2|PPFD-benchmark-Phloem-EX-leaf.xls-20230728-20:48.xlsx
PPFD: 50.0 CO2 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 50.0 CO2 flux: 0.7007744983326705  RBPC_M cell flux: 0.8348404933405801
PPFD: 50.0 CO2 flux: 0.7007744983330566  RBPC_M cell flux: 0.8348404933344213
PPFD: 50.0 CO2 flux: 0.7007744983332614  RBPC_M cell flux: 0.8348404934555396
PPFD: 50.0 CO2 flux: 0.700774498333223  RBPC_M cell flux: 0.8348404933337892
PPFD: 50.0 CO2 flux: 0.700774498333219  RBPC_M cell flux: 0.8348404933349304
PPFD: 50.0 CO2 flux: 0.7007744983330177  RBPC_M cell flux: 0.8348404932875689
PPFD: 50.0 CO2 flux: 0.700774498332466  RBPC_M cell flux: 0.8348404933213969
PPFD: 50.0 CO2 flux: 0.7007744983332305  RBPC_M cell flux: 0.834840493335455
PPFD: 50.0 CO2 flux: 0.7007744983332361  RBPC_M cell flux: 0.8348404933294116
PPFD: 211.11111111111111 CO2 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 211.111111111

In [12]:
#For Mature Leaf (Phloem as Objective function)
#Wild Type

#Define n and ppfd ranges for simulating n and light conditions respectively
ppfd_range = np.linspace(50, 1500, 10) #array([ 250.,  500.,  750., 1000., 1250., 1500.])
co2_range = np.linspace(0,22, 10)
#First initialize excel spreadsheet
filepath = './flux_results/CO2_benchmark/FBA/Trans/'
filename = 'CO2|PPFD-benchmark-Phloem-EX-leaf-07-28-2023.xls'
workbook = mm.create_xlsx(filename, filepath)


for ppfd in ppfd_range:

    co2_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in wt_model.reactions))

    for co2 in co2_range:
        
        #Generate instance of model every iteration
        with trans_model as m1:

            m1.medium = define_model_medium(m1, co2=co2, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf) #initialize model
            turn_off_cofac_cycles(m1)
            add_tissue_constraints(m1)
            add_enzyme_constraints(m1)
            add_ngam_cons(m1, ppfd)                #Set ATP NGAM value to linear PPFD formula
            add_trans_constraints(m1)
            
            #Change objective function to mature leaf
            m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
            m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
            m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

            #Optimize then check optimality
            while True:
                solution = m1.optimize()
                if solution.status == 'optimal':
                    sample_fluxes = cobra.flux_analysis.pfba(m1).fluxes
                    #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                    sample_fluxes = cobra.flux_analysis.loopless_solution(m1, sample_fluxes).to_frame()
                    print("PPFD:", ppfd, "CO2 flux:", sample_fluxes['fluxes']['CO2tex_M'], " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                    #Append to Dataframe
                    co2_solns_2cell_fba[co2] = sample_fluxes['fluxes']
                    break
                else:
                    continue
                
    #Write matrix to excel workbook
        
    co2_solns_2cell_fba.to_excel(workbook, sheet_name=str(ppfd))

workbook.close()

#Notes:
#Complete dataframe does not save
#I think I fixed it now, I saved each instance to the wrong column lol



Successfully generated CO2|PPFD-benchmark-Phloem-EX-leaf-07-28-2023.xls-20230728-20:53 in ./flux_results/CO2_benchmark/FBA/Trans/CO2|PPFD-benchmark-Phloem-EX-leaf-07-28-2023.xls-20230728-20:53.xlsx
PPFD: 50.0 CO2 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 50.0 CO2 flux: 0.7344094893203443  RBPC_M cell flux: 0.8788133414549149
PPFD: 50.0 CO2 flux: 0.7344094893203402  RBPC_M cell flux: 0.8788133414500348
PPFD: 50.0 CO2 flux: 0.7344094893203457  RBPC_M cell flux: 0.8788133414549191
PPFD: 50.0 CO2 flux: 0.7344094893203406  RBPC_M cell flux: 0.8788133414577001
PPFD: 50.0 CO2 flux: 0.7344094893203403  RBPC_M cell flux: 0.8788133414556235
PPFD: 50.0 CO2 flux: 0.7344094893203498  RBPC_M cell flux: 0.8788133414558675
PPFD: 50.0 CO2 flux: 0.7344094893203482  RBPC_M cell flux: 0.8788133414608515
PPFD: 50.0 CO2 flux: 0.7344094893203633  RBPC_M cell flux: 0.878813341449554
PPFD: 50.0 CO2 flux: 0.7344094893203398  RBPC_M cell flux: 0.8788133414567382
PPFD: 211.11111111111111 CO2 flux: 0.0  RBPC_M cell f

In [23]:
#Trans PPFD/CO2

#Define n and ppfd ranges for simulating n and light conditions respectively
ppfd_range = np.linspace(25, 1500, 10) #array([ 250.,  500.,  750., 1000., 1250., 1500.])
co2_range = np.linspace(0,22.2, 10)
#First initialize excel spreadsheet
filepath = './flux_results/CO2_benchmark/FBA/Trans/'
filename = 'CO2-PPFD-benchmark-Trans-Phloem.xls'
workbook = mm.create_xlsx(filename, filepath)


for ppfd in ppfd_range:

    co2_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in trans_model.reactions))

    for co2 in co2_range:
        
        #Generate instance of model every iteration
        with trans_model as m1:

            m1.medium = define_model_medium(m1, co2=co2, o2=inf, ppfd=ppfd, h=inf, nh4=inf, no3=inf) #initialize model
            turn_off_cofac_cycles(m1)
            add_tissue_constraints(m1)
            add_enzyme_constraints(m1)
            add_ngam_cons(m1, ppfd)                #Set ATP NGAM value to linear PPFD formula

            #Change objective function to mature leaf
            m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
            m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
            m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

            #Optimize then check optimality
            while True:
                solution = m1.optimize()
                if solution.status == 'optimal':
                    sample_fluxes = cobra.flux_analysis.pfba(m1).fluxes
                    #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                    sample_fluxes = cobra.flux_analysis.loopless_solution(m1, sample_fluxes).to_frame()
                    print("PPFD:", ppfd, "CO2 flux:", sample_fluxes['fluxes']['CO2tex_M'], " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                    #Append to Dataframe
                    co2_solns_2cell_fba[co2] = sample_fluxes['fluxes']
                    break
                else:
                    continue
                
    #Write matrix to excel workbook
        
    co2_solns_2cell_fba.to_excel(workbook, sheet_name=str(ppfd))

workbook.close()

#Notes:
#Complete dataframe does not save
#I think I fixed it now, I saved each instance to the wrong column lol



Successfully generated CO2-PPFD-benchmark-Trans-Phloem.xls-20230316-03:53 in ./flux_results/CO2_benchmark/FBA/Trans/CO2-PPFD-benchmark-Trans-Phloem.xls-20230316-03:53.xlsx
PPFD: 25.0 CO2 flux: 0.0  RBPC_M cell flux: 0.0
PPFD: 25.0 CO2 flux: 0.09041830695731458  RBPC_M cell flux: 0.10780211170149974
PPFD: 25.0 CO2 flux: 0.09041830695731107  RBPC_M cell flux: 0.10780211170144816
PPFD: 25.0 CO2 flux: 0.09041830695732679  RBPC_M cell flux: 0.09620352321739563
PPFD: 25.0 CO2 flux: 0.0904183069573251  RBPC_M cell flux: 0.10780211170148626
PPFD: 25.0 CO2 flux: 0.09041830695733472  RBPC_M cell flux: 0.10780211170208821
PPFD: 25.0 CO2 flux: 0.09041830695732515  RBPC_M cell flux: 0.09620352321815423
PPFD: 25.0 CO2 flux: 0.09041830695731455  RBPC_M cell flux: 0.10780211170146035
PPFD: 25.0 CO2 flux: 0.09041830695730158  RBPC_M cell flux: 0.10780211170145429
PPFD: 25.0 CO2 flux: 0.09041830695731698  RBPC_M cell flux: 0.09614158609601127
PPFD: 188.88888888888889 CO2 flux: 0.0  RBPC_M cell flux: 0.0

In [ ]:
#Let's graph the initial results first

#plot PPFD vs CO2 uptake

phloem_trans_out = ppfd_2cell_trans_phloem.loc[['DM_Phloem_BS']].values.flatten()
biomass_trans_M = ppfd_2cell_trans_bm.loc[['Straw_Biomass_M']].values.flatten()
biomass_trans_BS = ppfd_2cell_trans_bm.loc[['Straw_Biomass_BS']].values.flatten()

phloem_out = ppfd_2cell_wt_phloem.loc[['DM_Phloem_BS']].values.flatten()
biomass_out_M = ppfd_2cell_wt_bm.loc[['Straw_Biomass_M']].values.flatten()
biomass_out_BS = ppfd_2cell_wt_bm.loc[['Straw_Biomass_BS']].values.flatten()
ppfd_e = list(ppfd_2cell_wt_phloem.columns)

plt.plot(ppfd_e, phloem_out, label="Phloem out")
plt.plot(ppfd_e, biomass_out_M, label="Biomass out (M cell)")
plt.plot(ppfd_e, biomass_out_BS, label="Biomass out (BS cell)")
plt.title(label='Photoassimilate flux rates for Growing and Mature Leaf')
plt.xlabel('PPFD (umol/m2 s)')
plt.ylabel('Reaction flux (umol/m2 s)')
plt.legend()
plt.show()



#I think I need to graph previous data regarding this?
#Based on this, it shows that photoassimilation becomes unstable in intermediate lights  as opposed to
#Having a more stable output 

In [ ]:
plt.plot(ppfd_e, phloem_out, label="Phloem out(WT)")
plt.plot(ppfd_e, phloem_trans_out, label="Phloem out (Trans)")
plt.title(label='Photoassimilate flux rates for WT and trans')
plt.xlabel('PPFD (umol/m2 s)')
plt.ylabel('Reaction flux for Phloem sap (umol/m2 s)')
plt.legend()
plt.show()



In [ ]:
#Get list of active reactions per PPFD

Initial thoughts:
- Based on the Light scan experiments above we can see quite a greater variation in terms of Phloem output between flux ranges 200 to 500, and we can also see a minor increase in Rubisco activity in the BS cells. However, I think this still needs to be tested via flux sampling to determine the actual range.
- A similar dip exists even in the Wild type mature leaf but generally stabilizes faster compared to the transgenic rice. 
- I believe this is linked to ATP balancing and the transition from Mitochondrial ATP production to stromal ATP production.

- FUrthermore, I've observed that Rubisco activiity is also limited by Formate Dehydrogenase activity. When it is closed, Rubisco activity is only limited to around ~40 umol m2/s flux values (30 vc + 10 vo) while increasing the upper bound of FDHNc to around 25 allows it to have a ceiling of ~80 umol m2/s, which is around the experimental Rubisco activity measured by Ermakova et al (2021)
    - However, turning this reaction on causes the model to produce ATP with a flux rate that's almost x2 that of previous modelling attempts as checked from Shameer et al (2018, 2021). Furthermore, in Shameer et al (2021) the ranges for Rubisco carboxylation plays around 28-31, which is the maximum rate observed in this experiment. I think this would be a more viable route if ever.
    - Furthermore, based on Shameer et al's hybrid model and flux distributions the ATPase flux values are quite concordant with each other.
    - Lastly, the results pertaining to ATP synthesis is quite concordant with the results obtained by Shameer et al in their 2018 publication wherein mito-stromal ATP synthase shift was also observed.
    
    - Weirdly enough I'm observing that the addition of the trans constraints caused the flow of CO2 directly from the M cell to the BS cell at moderate light conditions, as opposed to the WT. Furthermore, more background flux is observed in all cases in the transgenic 2 cell. 
        - I need to check whether the absolute fluxes actually varied in these cases.
    
    - In both WT and Transgenic models Pyruvate is the predominant flux at low to medium light levels, while in the Transgenic model there is actual Gas flux from M to BS cells (both CO2 and O2)
    - In the transgenic model some flux of Glycolate from the M cell is detected in high light conditions (>750 ppfd) while some backflow of g3p from the BS cell to the M cell is observed.
    - PPFD to CO2 assimilation is much more steeper than what I've obtained in the past. I wonder why? I'm not too sure about it either
        - I think it is because I was only inputting NO3 into the mix rather than both NH3/NO3, as well as having free H+ import, which I closed. 
            - Upon testing this, I've observed that limiting H+ import now prevents that sharp increase in CO2 assimilation, with Max A approximately around 800 ppfd. Furthermore, an infeasible solution is now obtained for light intensities below 50 umol m2/s owing to the lack of source for proton demand. 
            - Adjusting this, an A roughly equivalent to the measured dark respiration values measured by Ermakova et al (2021)  is now obtained around 50 ppfd (Which, conversely, is the threshold designated by Topfer et al (2020) to delineate light compensation.
                - Maybe I can design an experiment to find the minimum value where the solver can find a feasible solution, then compare that to the trans one?
            -Adjusting the H+ demand (by allowing H+ import and varying import rates) actually removes the inverse relationship between Mitochondrial and Stromal ATP synthase flux. What gives?
                - I think modelling artifact yun since it allows system to simply import yung surplus H+ nila even though it could be supplied by photosynthesis (H2O  -> 2H2 + O2 based nga dun sa light reaction stoichiometry)
    

#Notes: Nov 26, 2022
I've revised the reaction for PEPC which I've changed from CO2 +H2O + PEP <-> Pi + H+ + OAA to HCO3 + PEP <-> Pi + OAA

In [ ]:
#Other artifacts as of the moment include the following:
#model doesn't produce any Oxygen. Maybe I should add a specific demand reaction for it?
#Based on FVA, the model now produces some oxygen via the demand reaction. 

In [ ]:
#Model also facilitates transfer of metabolites unlike before.

In [ ]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.